In [ ]:
!pip uninstall h5py
!pip install h5py==2.10.0

Found existing installation: h5py 2.10.0
Uninstalling h5py-2.10.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/h5py-2.10.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/h5py/*
Proceed (y/n)? y
  Successfully uninstalled h5py-2.10.0
  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install face_detection

In [ ]:
import os
import numpy as np
import cv2
import face_detection 
from sklearn.cluster import DBSCAN
from keras.models import load_model
from keras.applications.resnet50 import preprocess_input
import tqdm

Using TensorFlow backend.


In [ ]:
# Path to the Working Environment

# On a Local Environment => set BASE_PATH  = ""
BASE_PATH = "/content/drive/MyDrive/ML/"

# Path to Input Video File in the BASE_PATH
FILE_PATH = "video1.mp4"

In [ ]:
# Initialize a Face Detector 
# Confidence Threshold can be Adjusted, Greater values would Detect only Clear Faces

detector = face_detection.build_detector("DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3)

In [ ]:
# Load Pretrained Face Mask Classfier (Keras Model)

mask_classifier = load_model("/content/drive/MyDrive/ML/Weights_Models/ResNet50_Classifier_New.h5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [ ]:
# Set the Safe Distance in Pixel Units (Minimum Distance Expected to be Maintained between People)
# This Parameter would Affect the Results, Adjust according to the Footage captured 

threshold_distance = 150